# Get Started

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Kqlmagic:__ 0.1.90<br>
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  Log Analytics - SecurityEvent
    
### Description
The notebook provides you basic knowledge to use Azure Sentinel Notebooks.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Retrieve Log Analytics information 
2. Install Kqlmagic
3. Log into Log Analytics
4. Get a list of available data tables from Log Analytics
5. Run a simple Kusto query

## 1. Retrieve Log Analytics Information

In [ ]:
path = %env PATH
dsvm = False
if '/dsvm/' in path:
    dsvm = True
    
if dsvm == False:
    # Run this if you are using Free Compute
    from utils import config_reader as mod
    workspace_id = mod.config_reader.read_config_values("config.json")[3]
    tenant_id = mod.config_reader.read_config_values("config.json")[0]

    ######################################################################
    your_workspace_name = mod.config_reader.read_config_values("config.json")[4]
    print("Your Log Analytic Workspace: " + your_workspace_name)
    ######################################################################
else:
    # Run this if you are using DSVM.  You need to copy the values from config.json, if the file has no value, then you need to go to Log Analytics Portal to get the information.
    tenant_id = input('tenant_id:')
    subscription_id = input('subscription_id:')
    resource_group = input('resource_group:')
    workspace_id = input('workspace_id:')
    workspace_name = input('workspace_name:')

## 2. Install Required Packages
These are the two core packages required by most Azure Sentinel Notebooks. You need to install these only once in each Azure Notebooks project (or other Python environment) but they are both updated regularly so it is worth running<br>
```pip install {pkg} --upgrade```<br>
every few weeks to get the latest features and fixes.

In [ ]:
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

### 2.1 Install KqlMagic 
For more information and sample notebooks go to [Kqlmagic github](https://github.com/Microsoft/jupyter-Kqlmagic)

In [ ]:
# run only once unless upgrade
!pip install Kqlmagic --upgrade

In [ ]:
# Make sure you have 0.1.90 or above, if not, run Kqlmagic installation again
%reload_ext Kqlmagic

If you need any help for kql magic     --> %kql --help "help" -popup_window
If you need any help for Log Analytics --> %kql --help "LogAnalytics"

### 2.2 Configure Azure Notebooks Project to use Python 3.6 by Default
Unless it is configured, Azure notebooks will default to Python 3.5. Kqlmagic and the msticpy package both need Python 3.6 or later. If you do not configure a project default you will have to manually set this in the notebook every time you load a notebook.

Follow the guidance in the *HowTos* folder - [Azure Notebooks - Configure Python Version](./HowTos/AzureNotebooks-ConfigurePythonVersion.ipynb)

## 3. Log into Log Analytics

In [ ]:
# Want to know more about Log Analytics Authentication? Please run this cell, then click the link below
from IPython.display import HTML
HTML('<a href="https://github.com/Azure/Orion/blob/master/Notebooks/Help.md" target="_blank")>Details about Log Analytics Authentication</a>')

In [ ]:
# You must run this cell to log into Log Analytics to continue
%kql loganalytics://code;workspace=workspace_id;tenant=tenant_id;alias="SentinelDB"

## 4. Get a list of available data tables from Log Analytics

1. Tables with records

In [ ]:
%kql search * | summarize count() by Type

2. All Tables in drop down

In [ ]:
import ipywidgets as widgets
from IPython.display import display
dbSchema = %kql --schema "SentinelDB@loganalytics"
tables = list(dbSchema.keys())
selected_table = widgets.Dropdown(options=tables, value=tables[1],description='Data Table:')
display(selected_table)

3. Choose columns from the selected table

In [ ]:
columns = list(dbSchema[selected_table.value])
columns.sort()
selected_columns = widgets.SelectMultiple(options=columns, value=[],description='Data Columns:')
display(selected_columns)

4. List unique values for selected columns

In [ ]:
%kql {selected_table.value} | where TimeGenerated >= ago(30d) | take 1000
if len(_kql_raw_result_) > 0:
    df = _kql_raw_result_.to_dataframe()
else:
    df = None
    print('no data')

In [ ]:
for index, column in enumerate(selected_columns.value):
    unique_values = df[column].drop_duplicates().sort_values()
    values_dropdown = widgets.Dropdown(options=unique_values, description=column)
    display(values_dropdown)

5. Search a term

In [ ]:
# scope to a table and a column
%kql search in ({selected_table.value}) {selected_columns.value[0]}: '*local service' | take 5

## 5. Run a simple Kusto query

In [ ]:
%kql SecurityEvent | take 5 | project Account, Computer
df = _kql_raw_result_.to_dataframe()
df